In [33]:
import pandas as pd
from pathlib import Path
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI

In [34]:
import os

base_path = "/Users/gourav/Documents/ProjectsRepos/AI/Metadata/Docs"
filepath = f"{base_path}/CHATGPT_API_KEY.txt"
with open(filepath, "r") as f:
  api_key = ' '.join(f.readlines())
  os.environ["OPENAI_API_KEY"] = api_key
  os.environ["GOOGLE_API_KEY"] = api_key

In [35]:
import pandas as pd
from pathlib import Path

data_dir = Path("/Users/gourav/Documents/ProjectsRepos/AI/Metadata/Docs")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

# --------------

# import pandas as pd
# from pathlib import Path

# data_dir = Path("/Users/gourav/Documents/ProjectsRepos/AI/Metadata/Docs")
# csv_files = sorted([f for f in data_dir.glob("*.csv")])
# dfs = []

# # Define a function to read and process chunks
# def process_chunks(chunk):
#     # Append the chunk to the list of dataframes
#     dfs.append(chunk)

# # Read the first 1000 entries from each CSV file in chunks
# for csv_file in csv_files:
#     print(f"processing file: {csv_file}")
#     try:
#         # Read the CSV file in chunks of 1000 rows
#         chunk_iter = pd.read_csv(csv_file, chunksize=1000)
#         # Process each chunk
#         for chunk in chunk_iter:
#             process_chunks(chunk.head(1000))  # Extract the first 1000 entries from each chunk
#     except Exception as e:
#         print(f"Error parsing {csv_file}: {str(e)}")

# # Concatenate all dataframes
# final_df = pd.concat(dfs, ignore_index=True)

# Print the first few rows of the concatenated dataframe
# print(final_df.head())

processing file: /Users/gourav/Documents/ProjectsRepos/AI/Metadata/Docs/lei.csv
processing file: /Users/gourav/Documents/ProjectsRepos/AI/Metadata/Docs/relation__cityGroup.csv


In [36]:
tableinfo_dir = "/Users/gourav/Documents/ProjectsRepos/AI/Metadata/CSVTableInfo"
!mkdir {tableinfo_dir}

mkdir: /Users/gourav/Documents/ProjectsRepos/AI/Metadata/CSVTableInfo: File exists


In [37]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )

# class TableInfo(BaseModel):
#     """Information regarding a structured table."""
    
#     table_name: str = Field(
#         ..., description="table name (must be underscores and NO spaces)"
#     )
#     table_summary: str = Field(
#         ..., description="short, concise summary/caption of the table"
#     )
#     foreign_key: str = Field(
#         ..., description="Use the column that is common in tables and create foreign key mapping between two tables"
#     )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise. 
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-3.5-turbo"),
    prompt_template_str=prompt_str,
)

In [39]:
import json

def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )

table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)

Processed table: LegalEntityInfo
Processed table: RelationshipInfo


In [61]:
table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    print(table_info)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
        table_infos.append(table_info)

table_name='LegalEntityInfo' table_summary='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated entities, successor entities, legal entity events, and conformity flag.'
table_name='RelationshipInfo' table_summary="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, periods, qualifiers, quantifiers, and registration details. Relationship_EndNode_NodeID is the foreign key in the table Relationship_Information. LEI column is the primary key in the table Entity_Legal_Info which is the foreign key Relationship_EndNode_NodeID column in the table Relationship_Information. Relationship_RelationshipType column contains the information about the relation between the given company name. There are 4 type of relations in Relationship_RelationshipType column 'IS_DIRECTLY_CONSOLIDATED_BY', 'IS_ULTIMATELY_CONSOLIDATED_BY', 'IS_FU

In [62]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re
import pandas as pd
from tqdm import tqdm

# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        total_rows = len(df)
        with tqdm(total=total_rows, desc=f"Inserting data into {table_name}") as pbar:
            for _, row in df.iterrows():
                insert_stmt = table.insert().values(**row.to_dict())
                conn.execute(insert_stmt)
                pbar.update(1)

        conn.commit()

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# Assuming dfs is a list of DataFrames
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    table_name = tableinfo.table_name if tableinfo else f"Table_{idx}"
    print(f"Creating table: {table_name}")
    create_table_from_dataframe(df, table_name, engine, metadata_obj)
    # Print DataFrame shape
    print(f"DataFrame shape for {table_name}: {df.shape}")

Creating table: LegalEntityInfo


Inserting data into LegalEntityInfo: 100%|████████| 1/1 [00:00<00:00, 65.80it/s]


DataFrame shape for LegalEntityInfo: (1, 338)
Creating table: RelationshipInfo


Inserting data into RelationshipInfo: 100%|█| 798/798 [00:00<00:00, 2217.51it/s]

DataFrame shape for RelationshipInfo: (798, 29)


In [63]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)
table_schema_objs = []
table_node_mapping = SQLTableNodeMapping(sql_database)

# table_schema_objs = [
#     SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
#     for t in table_infos
# ]  # add a SQLTableSchema for each table

for idx in range(len(table_infos)):
    table_data = _get_tableinfo_with_index(idx)
    table_schema_objs.append(SQLTableSchema(table_name=table_data.table_name, context_str=table_data.table_summary))

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [64]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)
table_schema_objs = []
table_node_mapping = SQLTableNodeMapping(sql_database)
    
# table_schema_objs = [
#     SQLTableSchema(table_name=t[tindex].table_name, context_str=_get_tableinfo_with_index(tindex).table_summary)
#     for tindex, t in table_infos
# ]  # add a SQLTableSchema for each table

for idx in range(len(table_infos)):
   table_data = _get_tableinfo_with_index(idx)
   table_schema_objs.append(SQLTableSchema(table_name=table_data.table_name, context_str=table_data.table_summary))


obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=2)

In [65]:
table_schema_objs

[SQLTableSchema(table_name='LegalEntityInfo', context_str='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated entities, successor entities, legal entity events, and conformity flag.'),
 SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, periods, qualifiers, quantifiers, and registration details. Relationship_EndNode_NodeID is the foreign key in the table Relationship_Information. LEI column is the primary key in the table Entity_Legal_Info which is the foreign key Relationship_EndNode_NodeID column in the table Relationship_Information. Relationship_RelationshipType column contains the information about the relation between the given company name. There are 4 type of relations in Relationship_RelationshipType column 'IS_DIRECTLY_CONSOLIDATED_BY', 'IS_ULT

In [66]:
table_infos

[TableInfo(table_name='LegalEntityInfo', table_summary='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated entities, successor entities, legal entity events, and conformity flag.'),
 TableInfo(table_name='RelationshipInfo', table_summary="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, periods, qualifiers, quantifiers, and registration details. Relationship_EndNode_NodeID is the foreign key in the table Relationship_Information. LEI column is the primary key in the table Entity_Legal_Info which is the foreign key Relationship_EndNode_NodeID column in the table Relationship_Information. Relationship_RelationshipType column contains the information about the relation between the given company name. There are 4 type of relations in Relationship_RelationshipType column 'IS_DIRECTLY_CONSOLIDATED_BY', 'IS_ULTIMATEL

In [67]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

In [68]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.llms import ChatResponse


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [69]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [70]:
llm = OpenAI(model="gpt-3.5-turbo")

In [83]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [84]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [85]:
from llama_index.core import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
import os
from pathlib import Path
from typing import Dict


def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "table_index_dir"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""
    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")
        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:
                cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))

            # index each row, put into vector store index
            nodes = [TextNode(text=str(t)) for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index"
            )
        vector_index_dict[table_name] = index

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: LegalEntityInfo
Indexing rows in table: RelationshipInfo


In [100]:
test_retriever = vector_index_dict["RelationshipInfo"].as_retriever(
    similarity_top_k=2
)

In [102]:
nodes = test_retriever.retrieve("CITIGROUP INC")
print(nodes[1].get_content())

('54930004XKXOKX76FG65', 'LEI', '6SHGI4ZSSLCXXQSBB395', 'LEI', 'IS_ULTIMATELY_CONSOLIDATED_BY', 'ACTIVE', '2017-01-01T00:00:00.000Z', '2017-12-31T00:00:00.000Z', 'ACCOUNTING_PERIOD', '2017-10-02T00:00:00.000Z', None, 'RELATIONSHIP_PERIOD', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '2016-05-05T01:55:00.000Z', '2023-07-31T17:34:15.017Z', 'LAPSED', '2019-07-19T00:33:00.000Z', '5493001KJTIIGC8Y1R12', 'PARTIALLY_CORROBORATED', 'SUPPORTING_DOCUMENTS', None)


In [103]:
test_retriever = vector_index_dict["LegalEntityInfo"].as_retriever(
    similarity_top_k=1
)

In [104]:
nodes = test_retriever.retrieve("City group inc")
print(nodes[0].get_content())

('6SHGI4ZSSLCXXQSBB395', 'CITIGROUP INC.', 'en', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'en', 'CORPORATION TRUST CENTER 1209 ORANGE ST', None, None, 'c/o THE CORPORATION TRUST COMPANY', None, None, None, 'WILMINGTON', 'US-DE', 'US', 19801, 'en', '388 GREENWICH STREET', None, None, None, None, None, None, 'NEW YORK', 'US-NY', 'US', 10013, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [91]:
response = qp.run(
    query="Write deatils of LEI is 6SHGI4ZSSLCXXQSBB395 and it's direct childern legal entity?"
)
print(str(response))

> Running module input with input: 
query: Write deatils of LEI is 6SHGI4ZSSLCXXQSBB395 and it's direct childern legal entity?

> Running module table_retriever with input: 
input: Write deatils of LEI is 6SHGI4ZSSLCXXQSBB395 and it's direct childern legal entity?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='LegalEntityInfo', context_str='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated ...

> Running module text2sql_prompt with input: 
query_str: Write deatils of LEI is 6SHGI4ZSSLCXXQSBB395 and it's direct childern legal entity?
schema: Table 'LegalEntityInfo' has columns: LEI (VARCHAR), Entity_LegalName (VARCHAR), Entity_LegalName_xmllang (VARCHAR), Entity_OtherEntityNames_OtherEntityName_1 (INTEGER), Entity_OtherEntityNames_OtherEn...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically

In [53]:
response = qp.run(
    query="ultimate Childern legal entity of 6SHGI4ZSSLCXXQSBB395"
)
print(str(response))

> Running module input with input: 
query: ultimate Childern legal entity of 6SHGI4ZSSLCXXQSBB395

> Running module table_retriever with input: 
input: ultimate Childern legal entity of 6SHGI4ZSSLCXXQSBB395

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='LegalEntityInfo', context_str='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated ...

> Running module text2sql_prompt with input: 
query_str: ultimate Childern legal entity of 6SHGI4ZSSLCXXQSBB395
schema: Table 'LegalEntityInfo' has columns: LEI (VARCHAR), Entity_LegalName (VARCHAR), Entity_LegalName_xmllang (VARCHAR), Entity_OtherEntityNames_OtherEntityName_1 (INTEGER), Entity_OtherEntityNames_OtherEn...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answ

In [54]:
response = qp.run(
    query="How many childern companies CITIGROUP INC has?"
)
print(str(response))

> Running module input with input: 
query: How many childern companies CITIGROUP INC has?

> Running module table_retriever with input: 
input: How many childern companies CITIGROUP INC has?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: How many childern companies CITIGROUP INC has?
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the re

In [55]:
response = qp.run(
    query="What are the details of the LEI 6SHGI4ZSSLCXXQSBB395 and its related legal entity?"
)
print(str(response))

> Running module input with input: 
query: What are the details of the LEI 6SHGI4ZSSLCXXQSBB395 and its related legal entity?

> Running module table_retriever with input: 
input: What are the details of the LEI 6SHGI4ZSSLCXXQSBB395 and its related legal entity?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='LegalEntityInfo', context_str='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated ...

> Running module text2sql_prompt with input: 
query_str: What are the details of the LEI 6SHGI4ZSSLCXXQSBB395 and its related legal entity?
schema: Table 'LegalEntityInfo' has columns: LEI (VARCHAR), Entity_LegalName (VARCHAR), Entity_LegalName_xmllang (VARCHAR), Entity_OtherEntityNames_OtherEntityName_1 (INTEGER), Entity_OtherEntityNames_OtherEn...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically co

In [105]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

In [106]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [107]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [108]:
response = qp.run(
    query="What is the LEI for CITYGROUP?"
)
print(str(response))

> Running module input with input: 
query: What is the LEI for CITYGROUP?

> Running module table_retriever with input: 
input: What is the LEI for CITYGROUP?

> Running module table_output_parser with input: 
query_str: What is the LEI for CITYGROUP?
table_schema_objs: [SQLTableSchema(table_name='LegalEntityInfo', context_str='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated ...

> Running module text2sql_prompt with input: 
query_str: What is the LEI for CITYGROUP?
schema: Table 'LegalEntityInfo' has columns: LEI (VARCHAR), Entity_LegalName (VARCHAR), Entity_LegalName_xmllang (VARCHAR), Entity_OtherEntityNames_OtherEntityName_1 (INTEGER), Entity_OtherEntityNames_OtherEn...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results 

In [111]:
response = qp.run(
    query="How many relation are ultimate child of 6SHGI4ZSSLCXXQSBB395?"
)
print(str(response))

> Running module input with input: 
query: How many relation are ultimate child of 6SHGI4ZSSLCXXQSBB395?

> Running module table_retriever with input: 
input: How many relation are ultimate child of 6SHGI4ZSSLCXXQSBB395?

> Running module table_output_parser with input: 
query_str: How many relation are ultimate child of 6SHGI4ZSSLCXXQSBB395?
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: How many relation are ultimate child of 6SHGI4ZSSLCXXQSBB395?
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntac

In [112]:
response = qp.run(
    query="How many relation are ultimate child of CITIGROUP INC?"
)
print(str(response))

> Running module input with input: 
query: How many relation are ultimate child of CITIGROUP INC?

> Running module table_retriever with input: 
input: How many relation are ultimate child of CITIGROUP INC?

> Running module table_output_parser with input: 
query_str: How many relation are ultimate child of CITIGROUP INC?
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: How many relation are ultimate child of CITIGROUP INC?
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query

> Running module input with input: 
query: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?

> Running module table_retriever with input: 
input: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?

> Running module table_output_parser with input: 
query_str: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: How many relation are IS_ULTIMATELY_CONSOLIDATED_BY with CITIGROUP INC?
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given 

In [115]:
response = qp.run(
    query="how many relations are managing fund for CITIGROUP INC?"
)
print(str(response))

> Running module input with input: 
query: how many relations are managing fund for CITIGROUP INC?

> Running module table_retriever with input: 
input: how many relations are managing fund for CITIGROUP INC?

> Running module table_output_parser with input: 
query_str: how many relations are managing fund for CITIGROUP INC?
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: how many relations are managing fund for CITIGROUP INC?
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite q

In [116]:
response = qp.run(
    query="LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table"
)
print(str(response))


> Running module input with input: 
query: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table

> Running module table_retriever with input: 
input: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table

> Running module table_output_parser with input: 
query_str: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table
table_schema_objs: [SQLTableSchema(table_name='LegalEntityInfo', context_str='Table containing detailed information about a legal entity, including legal name, addresses, registration details, entity status, associated ...

> Running module text2sql_prompt with input: 
query_str: LEI column in Entity_Legal_Info table is associated with which foreign key in relationship_information table
schema: Table 'LegalEntityInfo' has columns: LEI (VARCHAR), Entity_LegalName (VARCHAR), Entity_LegalName_xmllang (VARCHAR), 

In [117]:
response = qp.run(
    query="how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_retriever with input: 
input: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input questio

In [118]:
response = qp.run(
    query="how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc

> Running module table_retriever with input: 
input: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: how many RelationshipType in Relationship Information Table are direct children of CityGroup Inc
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNod

In [119]:
response = qp.run(
    query="how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_retriever with input: 
input: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: how many records are IS_DIRECTLY_CONSOLIDATED_BY with CityGroup Inc
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input questio

In [120]:
response = qp.run(
    query="how many records are direct child of CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: how many records are direct child of CityGroup Inc

> Running module table_retriever with input: 
input: how many records are direct child of CityGroup Inc

> Running module table_output_parser with input: 
query_str: how many records are direct child of CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: how many records are direct child of CityGroup Inc
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then lo

In [121]:
esponse = qp.run(
    query="Please write names of direct childern of the CityGroup Inc"
)
print(str(response))

> Running module input with input: 
query: Please write names of direct childern of the CityGroup Inc

> Running module table_retriever with input: 
input: Please write names of direct childern of the CityGroup Inc

> Running module table_output_parser with input: 
query_str: Please write names of direct childern of the CityGroup Inc
table_schema_objs: [SQLTableSchema(table_name='RelationshipInfo', context_str="Summary of relationships between nodes with Relationship_StartNode_NodeID and Relationship_EndNode_NodeID, Relationship_RelationshipType, pe...

> Running module text2sql_prompt with input: 
query_str: Please write names of direct childern of the CityGroup Inc
schema: Table 'RelationshipInfo' has columns: Unnamed_0 (INTEGER), Relationship_StartNode_NodeID (VARCHAR), Relationship_StartNode_NodeIDType (VARCHAR), Relationship_EndNode_NodeID (VARCHAR), Relationship_End...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically corr